In [2]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')



c:\Users\cacic\anaconda3\envs\ldi2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\cacic\anaconda3\envs\ldi2\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cacic\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order t

In [3]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as file:
    grafo_json = json.load(file)
    
# Ahora `grafo_json` debería estar disponible para su uso en el bucle
nodos = []
for categoria, data in grafo_json.items():
    nodos.append(categoria)
    if 'subcategorias' in data:
        for subcategoria in data['subcategorias']:
            nodos.append(subcategoria)


In [4]:
# Generar los embeddings para los nodos
embeddings = model.encode(nodos)

In [5]:
# Crear índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # Usar L2 distance
index.add(np.array(embeddings))

In [6]:
# Función de consulta
def consulta_rag(consulta, index, nodos, model, k=5):
    consulta_embedding = model.encode([consulta])
    D, I = index.search(np.array(consulta_embedding), k=k)  # Buscar los 'k' nodos más cercanos
    
    print(f"Consulta: {consulta}")
    for i in range(k):
        print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")


In [7]:
# Crear una lista de nodos (incluyendo plagas y momentos de aplicación)
nodos = []

# Recorrer la jerarquía de datos (categorías -> productos -> plagas -> momentos de aplicación)
for cultivo, datos_cultivo in grafo_json.items():
    nodos.append(cultivo)  # Añadir el cultivo como nodo
    if 'Productos' in datos_cultivo:
        for producto, info_producto in datos_cultivo['Productos'].items():
            nodos.append(producto)  # Añadir el producto como nodo
            if 'Controla a' in info_producto:
                for plaga, detalles in info_producto['Controla a'].items():
                    nodos.append(plaga)  # Añadir la plaga como nodo
                    for momento in detalles['Momento de aplicacion']:
                        nodos.append(momento)  # Añadir el momento de aplicación como nodo


In [8]:
#Esta corrida demora aproximadamente 12 minutos

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Crear un modelo de embeddings (por ejemplo, Sentence-BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convertir los nodos a embeddings
nodos_embeddings = model.encode(nodos)

# Convertir los embeddings a un array NumPy
nodos_embeddings = np.array(nodos_embeddings)

# Guardar los embeddings como archivo .npy
np.save('nodos_embeddings.npy', nodos_embeddings)

# Crear un índice FAISS (usando el índice de búsqueda más cercano)
index = faiss.IndexFlatL2(nodos_embeddings.shape[1])  # Usar L2 para medir la distancia
index.add(nodos_embeddings)  # Agregar los embeddings al índice

# Guardar el índice FAISS en un archivo
faiss.write_index(index, 'nodos_index.faiss')

print("Embeddings y FAISS guardados exitosamente.")


Embeddings y FAISS guardados exitosamente.


In [15]:
# Función de consulta
def consulta_rag(consulta, index, nodos, model, k=10):
    consulta_embedding = model.encode([consulta])  # Codificar la consulta
    D, I = index.search(np.array(consulta_embedding), k)  # Buscar los k nodos más cercanos
    
    print(f"Consulta: {consulta}")
    for i in range(k):
        print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")

# Ejemplo de consulta
consulta_rag("Arroz", index, nodos, model)


Consulta: Arroz
Resultado 1: Arroz (Distancia: 0.0000)
Resultado 2: Arveja (Distancia: 0.6242)
Resultado 3: Maíz (Distancia: 1.0417)
Resultado 4: Avena (Distancia: 1.0425)
Resultado 5: Acelga (Distancia: 1.0491)
Resultado 6: Zarzamora (Distancia: 1.0529)
Resultado 7: Pomelo (Distancia: 1.0969)
Resultado 8: Almendro (Distancia: 1.1251)
Resultado 9: Garbanzo (Distancia: 1.1260)
Resultado 10: CZ 7905 IPRO I (Distancia: 1.1445)
